In [3]:
!pip install -U transformers


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/fc/04/0aad491cd98b09236c54ab849863ee85421eeda5138bbf9d33ecc594652b/transformers-4.36.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 4.8 MB/s eta 0:00:00
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/eb/3d/eee5f3c572a3f4db2ebabf5bd4f284f356078a5b5d27e6229b4450d5c5e4/tokenizers-0.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [1]:
from transformers import RobertaConfig, RobertaModel
import torch

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    top_k_top_p_filtering,
)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
print(device)

bnb_config = BitsAndBytesConfig(
    #load_in_4bit=True,
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload = True,
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_compute_dtype=torch.float16,
)

from transformers import AutoTokenizer, AutoModelForMaskedLM




tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                model_id,
                use_safetensors=True,
                quantization_config=bnb_config,
                trust_remote_code=True,
                device_map=device,
)

# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input.to(device))

/home/david/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.35s/it]


In [2]:
model.config

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": true,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4

In [7]:
from peft.tuners.lora import LoraLayer
from peft import LoraConfig, get_peft_model,prepare_model_for_kbit_training

# peft_config = LoraConfig(
#         r=16,  # dimension of the updated matrices
#         lora_alpha=32,  # parameter for scaling
#         target_modules= None,
#         lora_dropout=0.1,  # dropout probability for layers
#         bias="none",
#         task_type="CAUSAL_LM",
#         inference_mode = False,
#         # NAN's occur when included in training.. track down why....
#         #modules_to_save=['lm_head','embed_tokens'], #TODO check this out and see how I can use to just train the new tokens. 
#         #modules_to_save=['embed_tokens'], # Try just the emb tokens....
#         #modules_to_save=[],
#         # Could also try some other optimizers... maybe it is something to do with the WAdam build....
# )

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

ValueError: Please specify `target_modules` in `peft_config`

In [4]:
output = model(**encoded_input.to(device),
               output_hidden_states = True
              )
output.logits

tensor([[[-5.5625, -5.5820, -0.2493,  ..., -4.2031, -3.2617, -3.8809],
         [-5.6719, -5.8047, -4.2656,  ..., -2.7246, -1.0615, -2.4727],
         [-8.4844, -8.5469, -5.9180,  ..., -6.0781, -5.5898, -4.9297],
         ...,
         [-7.9141, -7.8008, -2.8691,  ..., -3.3281, -4.4961, -3.3047],
         [-7.8516, -8.2344, -2.9844,  ..., -6.6836, -7.3672, -4.8047],
         [-8.1016, -7.9062, -0.0510,  ..., -4.7734, -4.8984, -3.5781]]],
       device='cuda:0', grad_fn=<ToCopyBackward0>)

In [5]:
model.config

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-Instruct-v0.2",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": true,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4

In [ ]:


# if bnb_config is None:
#     bnb_config = BitsAndBytesConfig(
#         #load_in_4bit=True,
#         load_in_8bit=True,
#         llm_int8_enable_fp32_cpu_offload = True,
#         #bnb_4bit_quant_type="nf4",
#         #bnb_4bit_compute_dtype=torch.float16,
#     )

model_id = 't5-base'
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    use_safetensors=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map=device_map,
    token=access_token
)

tokenizer = AutoTokenizer.from_pretrained(model_id) #,token=access_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"



# try mistral 8x7

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

text = "Hello my name is"
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Try Phi-2

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)